In [0]:
import sys
if "pyodide" in sys.modules:
    import piplite
    await piplite.install('opvious>=0.14.0')

# Sudoku assistant

This notebook shows how we can use [mixed-integer programming](https://en.wikipedia.org/wiki/Integer_programming) to solve Sudoku grids and [Lagrangian relaxation](https://en.wikipedia.org/wiki/Lagrangian_relaxation) to automatically identify mistakes.

In [3]:
import opvious
import opvious.modeling as om
import pandas as pd

client = opvious.Client.default()

In [2]:
def pretty_grid(triples):
    """Transforms a list of triples into a natural Sudoku 2d grid"""
    positions = list(range(9))
    return (
        pd.DataFrame(triples, columns=['row', 'column', 'value'])
            .pivot_table(index='row', columns='column', values='value')
            .reindex(positions)
            .reindex(positions, axis=1)
            .applymap(lambda v: str(int(v)) if v == v else '')
    )

## Creating the model

We start by formulating Sudoku as an optimization problem using `opvious`' [declarative modeling API](https://opvious.readthedocs.io/en/stable/modeling.html).

In [3]:
class Sudoku(om.Model):
    positions = om.interval(0, 8, name='P')
    values = om.interval(1, 9, name='V')

    def __init__(self):
        self.input = om.Parameter.indicator(self.grid * self.values)
        self.output = om.Variable.indicator(self.grid * self.values)

    @property
    def grid(self):
        return self.positions * self.positions

    @om.constraint
    def output_matches_input(self):
        for i, j, v in self.grid * self.values:
            if self.input(i, j, v):
                yield self.output(i, j, v) >= self.input(i, j, v)

    @om.constraint
    def one_output_per_cell(self):
        for i, j in self.grid:
            yield om.total(self.output(i, j, v) == 1 for v in self.values)

    @om.constraint
    def one_value_per_column(self):
        for j, v in self.positions * self.values:
            yield om.total(self.output(i, j, v) == 1 for i in self.positions)

    @om.constraint
    def one_value_per_row(self):
        for i, v in self.positions * self.values:
            yield om.total(self.output(i, j, v) == 1 for j in self.positions)

    @om.constraint
    def one_value_per_box(self):
        for v, b in self.values * self.positions:
            yield om.total(
                self.output(3 * (b // 3) + c // 3, 3 * (b % 3) + c % 3, v) == 1
                for c in self.positions
            )
            
model = Sudoku()
model.specification()  # Generates the model's mathematical definitions

<div style="margin-top: 1em; margin-bottom: 1em;">
<details open>
<summary style="cursor: pointer; text-decoration: underline; text-decoration-style: dotted;">Sudoku</summary>
<div style="margin-top: 1em;">
$$
\begin{align*}
  \S^p_\mathrm{input}&: i \in \{0, 1\}^{P \times P \times V} \\
  \S^v_\mathrm{output}&: \omicron \in \{0, 1\}^{P \times P \times V} \\
  \S^a&: P \doteq \{ 0 \ldots 8 \} \\
  \S^a&: V \doteq \{ 1 \ldots 9 \} \\
  \S^c_\mathrm{outputMatchesInput}&: \forall p \in P, p' \in P, v \in V \mid i_{p,p',v} \neq 0, \omicron_{p,p',v} \geq i_{p,p',v} \\
  \S^c_\mathrm{oneOutputPerCell}&: \forall p \in P, p' \in P, \sum_{v \in V} \omicron_{p,p',v} = 1 \\
  \S^c_\mathrm{oneValuePerColumn}&: \forall p \in P, v \in V, \sum_{p' \in P} \omicron_{p',p,v} = 1 \\
  \S^c_\mathrm{oneValuePerRow}&: \forall p \in P, v \in V, \sum_{p' \in P} \omicron_{p,p',v} = 1 \\
  \S^c_\mathrm{oneValuePerBox}&: \forall v \in V, p \in P, \sum_{p' \in P} \omicron_{3 \left\lfloor \frac{p}{3} \right\rfloor + \left\lfloor \frac{p'}{3} \right\rfloor,3 \left(p \bmod 3\right) + p' \bmod 3,v} = 1 \\
\end{align*}
$$
</div>
</details>
</div>

## Finding solutions

Now that we've formulated the problem, we can use it to fill in Sudoku grids. We just need to pass in the initial values as `input` parameter and [start solving](https://opvious.readthedocs.io/en/stable/overview.html#solves).

In [4]:
async def fill_in(inputs):
    """Returns the triples representing a valid solution for the input triples"""
    response = await client.run_solve(
        specification=model.specification(),
        parameters={'input': inputs},
        assert_feasible=True,
    )
    output = response.outputs.variable('output')
    return list(output.index)

We test that it works on a simple example.

In [5]:
pretty_grid(await fill_in([
    (0, 0, 2),
    (1, 1, 4),
    (2, 2, 6),
]))

column,0,1,2,3,4,5,6,7,8
row,,,,,,,,,
0,2,5,8,4,7,1,6,9,3
1,3,4,9,5,2,6,8,1,7
2,1,7,6,3,9,8,4,2,5
3,8,6,3,1,5,2,9,7,4
4,7,1,5,9,3,4,2,6,8
5,4,9,2,6,8,7,5,3,1
6,5,3,7,8,6,9,1,4,2
7,6,8,4,2,1,3,7,5,9
8,9,2,1,7,4,5,3,8,6


## Identifying mistakes

Mistakes can happen when manually solving Sudoku puzzles. We often discover these much later, making it difficult to identify which decision(s) caused the grid to become infeasible.

Fortunately, we can also use our model to detect the _smallest set of changes_ needed to restore feasibility. We simply [relax the input matching constraint](https://opvious.readthedocs.io/en/stable/transformations.html#relaxing-constraints) and output the corresponding deficit. Clearing all cells with a deficit is guaranteed to make the grid solvable again.

In [6]:
async def find_mistakes(inputs):
    """Returns the smallest set of triples to make the input triples feasible."""
    response = await client.run_solve(
        specification=model.specification(),
        parameters={'input': inputs},
        transformations=[opvious.transformations.RelaxConstraints(['outputMatchesInput'])],
        assert_feasible=True,
    )
    deficit = response.outputs.variable('outputMatchesInput_deficit')
    return list(deficit.index)

We check that it works on a simple infeasible grid.

In [7]:
pretty_grid(await find_mistakes([
    (0, 0, 2),
    (1, 1, 2),  # Mistake
    (1, 7, 2),
]))

column,0,1,2,3,4,5,6,7,8
row,,,,,,,,,
0,,,,,,,,,
1,,2,,,,,,,
2,,,,,,,,,
3,,,,,,,,,
4,,,,,,,,,
5,,,,,,,,,
6,,,,,,,,,
7,,,,,,,,,
8,,,,,,,,,
